# k-nearest neighbour classification

This notebook contains a naive implementation of the k-nearest-neighbour (kNN) algorithm.  It is demonstrated with the IRIS data set.

In [ ]:
import numpy as np
import sklearn.datasets

from sklearn import model_selection

from matplotlib import pyplot as plt
%matplotlib inline

## Preparing the experiment

scikit-learn is a library for machine learning, which contains many standard algorithms and data sets.  We use it for loading the data set, saving writing our own parsing algorithms.


In [ ]:
ds = sklearn.datasets.load_iris()

In supervised learning we always have inputs $x_i$ which we want to map on targets $z_i$.  We save both data sets in numpy arrays ``AX`` and ``AZ``:

In [ ]:
AX, AZ = ds.data, ds.target

The ``.shape`` attribute of a numpy array gives us its size and form.

In [ ]:
print ('shape of input data:', AX.shape)
print ('shape of target data:', AZ.shape)

This means that we are dealing with 150 samples of data.  Each input has 4 different features.

To make sure we don't just "store" the data, we simulate "future data" by reserving a part of the data set for this.  scikit-learn provides us with the necessary functionality.

``StratifiedShuffleSplit`` mixes the data and separates them in a training set and a test set.  "Stratified" means that the number of samples per class remains constant.  If, for instance, we have a data set with 70% dogs and 30% cats, then also the training and validation sets have the approximate same distribution.

In [ ]:
splitter=model_selection.StratifiedShuffleSplit(n_splits=1,random_state=12)
train_idxs, test_idxs = list(splitter.split(AX,AZ))[0]
print("indices of the data for making the model:")
print(train_idxs)
print()
print("indices of the data for testing:")
print(test_idxs)

In [ ]:
# we can also give the slice operator a list of indices ``idxs``.
# Then a new array will be created which contains exactly the corresponding
# elements to which ``idxs`` points.

X, Z = AX[train_idxs], AZ[train_idxs]
TX, TZ = AX[test_idxs], AZ[test_idxs]

## Showing the training data

### Some statistics

In [ ]:
n_classes = np.unique(Z).size
print('number of classes:', n_classes)
print('numer of data points per class:', np.bincount(Z))
print('average input values:',  X.mean(0))
print('variances of the inputs', X.std(0))

### Scatterplot matrix

A scatterplot is useful to show high-dimensional data.  The horizontal axies shows histograms of the corresponding dimension.  Nondiagonal elements will be filled with scatterplots of the corresponding two dimensions.

Colouring is done following the respective classes.  This makes it immediately clear how easy or difficult the problem is.

Of course this approach is limited if information is divided amongst several dimensions.

In [ ]:
fig, axs = plt.subplots(n_classes, n_classes, squeeze=False, figsize=(12, 12))

for i in range(n_classes):
    for j in range(n_classes):
        if i == j:
            _ = axs[i, j].hist(X[:, i])
        else:
            axs[i, j].scatter(X[:, i], X[:, j], c=Z, lw=1, s=100, alpha=.7)

## Implementation kNN

### Finding the nearest neighbour

We start by computing the k nearest neighbours for some point ``query``.

For reasons of efficiency we don't use the points themselves, but their indices.

In [ ]:
def neighbour_idxs(data, query, k):
    """Gebe die ``k`` Indizes von ``data`` zurueck, welche am naechsten
    an ``query`` liegen."""
    # Wir berechnen die quadratische Distanz von jedem Punkt in ``data``
    # zu ``query``.
    # Auch wenn wir Euklidische Distanz annehmen, koennen wir auf das 
    # Ziehen der Wurzel verzichten, da es die Ordnung nicht veraendert.
    # data hat die shape (N, F) wo N die Anzahl der Samples in der Datenbank
    # ist und  F die Anzahl der Features; query hat Form (1, F)
    difference = data - query
    elem_sqrd_distance = difference ** 2
    sqrd_distance = elem_sqrd_distance.sum(1)
        
    # Wir sortieren das gesamte Array der Distanzen. Hier geht sicherlich
    # Performanz verloren da wir nur die kleinsten k benoetigen--aber eine
    # effiziente Implementation wuerde den Rahmen sprengen.
    sorted_idxs = np.argsort(sqrd_distance)
    
    # Hier benutzen wir den slice operator um nur die ersten k Elemente 
    # zurueckzugeben.
    k_idxs = sorted_idxs[:k]
    
    return k_idxs    

### Make a prediction

Next we make a function ``predict`` which determines a class according to the neighbours.

In [ ]:
def predict(data, targets, query, k):
    idxs = neighbour_idxs(data, query, k)
    
    # We use the slice operator here again.
    neighbour_targets = targets[idxs]
    
    # An dieser Stelle muessen wir die naechsten Nachbarn "abstimmen" lassen,
    # welche Klasse wir zurueckgeben. Beliebt ist die "majority vote", d.h. die
    # Klasse welche am meisten in den Nachbarn vorkommt ist die Ausgabe. Dazu
    # zaehlen wir die Klassen durch.
    # Die Funktion ``bincount`` von numpy kommt da gerade recht. Sie zaehlt die 
    # Anzahl vorkommen von jedem Integer der Reihe nach durch. 
    votes = np.bincount(neighbour_targets)
    return np.argmax(votes)

## Finding a best k

The correct choice of k is key for the final performance.  We call this a "hyperparameter".  How can we determine it here without using the test set again?

Idea: we repeat the split approach.  For each data point we use some candidates for k, if we take all other data into account.  This is called "leave-one-out cross-validation" or LOO or LOOCV.

Implemented in scikit-learn.

In [ ]:
def find_k(data, targets, candidates):
    """return that k which gives best kNN results according to LOO on the training data."""
    # We prepare an array filled with zeroes, counting the number of correct classifications.
    correct = np.zeros(len(candidates))
    
    loo = model_selection.LeaveOneOut()
    # Now iterate over each LOO split.
    for train_idxs, test_idxs in loo.split(data):
        # Here we iterate over all candidates for k.
        for j, cand in enumerate(candidates):
            prediction = predict(X[train_idxs], Z[train_idxs], X[test_idxs[0]], cand)
            if prediction == Z[test_idxs[0]]:
                # Increment if the prediction was correct.
                correct[j] += 1
    
    # Return that k for which we had a maximum number of correct predictions.
    return candidates[correct.argmax()]

In [ ]:
best_k = find_k(X, Z, range(1, 10))
print("Best k:", best_k)

## Testing the model

In [ ]:
predictions = np.array([predict(X, Z, TX[i], best_k) for i in range(TX.shape[0])])

In [ ]:
print('Accuracy:', (predictions == TZ).mean())

## Depict decision boundaries

We can look at the decision boundaries in which we plot the prediction of each
point in a raster.  This only really works well in 2D.

In [ ]:
# This code is partly taken from  
# http://scikit-learn.org/stable/auto_examples/neighbors/plot_classification.html#example-neighbors-plot-classification-py

h = .1
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))
zz = np.array([predict(X[:, :2], Z, np.array([x, y]), best_k) 
               for x, y in zip(xx.ravel(), yy.ravel())])

In [ ]:
fig, axs = plt.subplots(1, 1, squeeze=False, figsize=(9, 9))
ax = axs[0, 0]
ax.set_xlim(xx.min(), xx.max())
ax.set_ylim(yy.min(), yy.max())
ax.pcolormesh(xx, yy, zz.reshape(xx.shape), alpha=.9)
ax.scatter(TX[:, 0], TX[:, 1], c=TZ, s=100)

# Exercises

## 1. alternative distance function

In ``neigbour_idxs`` we assumed Euclidean distance:

$$
d_{\text{Euklid}}(x_i, x_j) = \sqrt{\sum_k (x_i^k - x_j^k)^2}
$$

An alternative would be the Manhattan distance:

$$
d_{\text{Manhattan}}(x_i, x_j) = \sum_k |x_i^k - x_j^k|
$$

**Exercise:** Implement the Manhattan distance in ``neighbour_idxs``. How does this effect the result?


## 2. Skewed data

Let's assume that the data is not scaled the same along each axis.  Please change the data in such a way that you multiply the first axis by 10, 100, or 1000.

**Exercise:** how does this affect the distance function?  How can you deal with this problem?
